# Data Analysis Notebook

This notebook demonstrates how to query the Medallion Architecture zones: **Raw**, **Curated**, and **Gold**.

In [ ]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
from pathlib import Path

# Manually defining paths here so this notebook can run standalone if needed,
# but ideally we import from config.py if the path is set correctly.
import config

: 

## 1. Helper Function

In [ ]:
def query_db(db_path, query):
    conn = sqlite3.connect(db_path)
    try:
        df = pd.read_sql(query, conn)
        return df
    finally:
        conn.close()

## 2. Gold Zone Analysis

Let's look at the aggregated metrics.

In [ ]:
# Top 5 Days by Revenue
query = """
SELECT * FROM sales_summary_daily 
ORDER BY total_sales DESC 
LIMIT 5
"""
df_gold = query_db(config.GOLD_DB_PATH, query)
df_gold

In [ ]:
# Plotting Daily Sales
query = "SELECT date, total_sales FROM sales_summary_daily ORDER BY date"
df_sales = query_db(config.GOLD_DB_PATH, query)
df_sales['date'] = pd.to_datetime(df_sales['date'])

plt.figure(figsize=(10, 5))
plt.plot(df_sales['date'], df_sales['total_sales'], marker='o', linestyle='-')
plt.title('Daily Sales Over Time')
plt.xlabel('Date')
plt.ylabel('Total Sales ($)')
plt.grid(True)
plt.xticks(rotation=45)
plt.show()

## 3. Product Performance

Which categories are generating the most revenue?

In [ ]:
query = "SELECT * FROM product_category_performance ORDER BY total_revenue DESC"
df_prod = query_db(config.GOLD_DB_PATH, query)
df_prod

In [ ]:
plt.figure(figsize=(10, 6))
plt.barh(df_prod['sub_category'], df_prod['total_revenue'], color='skyblue')
plt.xlabel('Total Revenue')
plt.title('Revenue by Sub-Category')
plt.gca().invert_yaxis()
plt.show()

## 4. Raw vs Curated Data Check

Comparing row counts to ensure data flowed correctly.

In [ ]:
raw_count = query_db(config.RAW_DB_PATH, "SELECT COUNT(*) as c FROM orders").iloc[0]['c']
curated_count = query_db(config.CURATED_DB_PATH, "SELECT COUNT(*) as c FROM orders").iloc[0]['c']

print(f"Raw Orders: {raw_count}")
print(f"Curated Orders: {curated_count}")